In [13]:
%matplotlib

import serial
import numpy as np
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import time
import threading
import queue
from collections import OrderedDict
import math

GPS_MULT = 10000000.0
M_PI = 3.14159265358979323846

def gps_destination(lat1, lon1, angle, distance_km):
    angle = np.radians(angle);
    dx = math.sin(angle) * distance_km;
    dy = math.cos(angle) * distance_km;

    cos = math.cos(lat1 * M_PI / 180.);
    cos2 = 2. * cos * cos - 1.;
    cos3 = 2. * cos * cos2 - cos;
    cos4 = 2. * cos * cos3 - cos2;
    cos5 = 2. * cos * cos4 - cos3;

    # multipliers for converting longitude and latitude
    # degrees into distance (http://1.usa.gov/1Wb1bv7)
    kx = (111.41513 * cos - 0.09455 * cos3 + 0.00012 * cos5);
    ky = (111.13209 - 0.56605 * cos2 + 0.0012 * cos4);

    lon2 = lon1 + dx / kx;
    lat2 = lat1 + dy / ky;
    
    return  lon2, lat2


patches = []

fig, ax = plt.subplots()
d_line, = plt.plot([0], [0], '-x')
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
plt.axis('equal')
plt.show()

def redraw(circles, lines):
    global patches
    global d_line
           
    for p in patches:
        p.remove()
    
    xmin = 9999999999999999
    ymin = 9999999999999999
    xmax = -9999999999999999
    ymax = -9999999999999999
    
    patches = []
    p = []
    i = 0
    for i in circles:
        c = circles[i]
        deg = (c['r'] / 1000.0)/111.0
        
        pts = []
        
        for angle in range(0, 360):
            pts.append(gps_destination(c['y'], c['x'], angle, (c['r'] / 1000.0)))
            
        circle = Polygon(pts, alpha=0.2)
        #circle = Circle((c['x'], c['y']), deg, alpha=0.2, ls='-', lw=1)
            
        patches.append(ax.add_patch(circle))
        
        xmin = min(c['x'], xmin)
        ymin = min(c['y'], ymin)
        xmax = max(c['x'], xmax)
        ymax = max(c['y'], ymax)

    
    linex = []
    liney = []
    for i in lines:
        l = lines[i]
        linex.append(l['x'])
        liney.append(l['y'])
        
        xmin = min(l['x'], xmin)
        ymin = min(l['y'], ymin)
        xmax = max(l['x'], xmax)
        ymax = max(l['y'], ymax)        
        
    d_line.set_data(linex, liney)
    
    pad = 0.1
    ax.set_xlim(xmin - pad, xmax + pad)
    ax.set_ylim(ymin - pad, ymax + pad)
    fig.canvas.draw()

def read_port():
    global done
    
    circles = OrderedDict()
    lines = OrderedDict()
    
    s = serial.Serial("/dev/ttyUSB0", 921600)
    done = False
    
    while (not done):
        while s.inWaiting() > 0:
            try:
                line = s.readline().decode("ascii") 
            except UnicodeDecodeError:
                continue

            print(line, end = '')
            if (line[0] in ["w", "c"]):
                data = line.split()
                i = int(data[1])
                if len(data) < 4:
                    continue
                
                try:
                    if data[0] == "c":
                        circle = {}
                        circle["y"] = int(data[2]) / GPS_MULT
                        circle["x"] = int(data[3]) / GPS_MULT
                        circle["r"] = int(data[4])
                        circles[i] = circle

                        line = {}
                        line["y"] = int(data[2]) / GPS_MULT
                        line["x"] = int(data[3]) / GPS_MULT
                        lines[i] = line                    

                    if data[0] == "w":
                        line = {}
                        line["y"] = int(data[2]) / GPS_MULT
                        line["x"] = int(data[3]) / GPS_MULT
                        lines[i] = line
                except:
                    pass

        redraw(circles, lines)
        plt.pause(0.05)
        
    s.close()
    print("Closed port")

def handle_close(evt):
    print('Closed Figure!')
    global done
    done = True

fig.canvas.mpl_connect('close_event', handle_close)
            
read_port()    

Using matplotlib backend: TkAgg
--waypoint_task_optimise_now()
c 0 481547833 169501500 500
c 1 481547833 169501500 500
c 2 483230666 171566000 500
c 3 481576000 173754166 500
c 4 482404500 174276000 500
c 5 483099000 174660500 500
c 6 484011666 175031333 500
c 7 481773000 175786666 500
Loading from SD 0
opti_angle 500.00
opti_latitude 481547833
opti_longtitude 169501500
opti_dist_m 0

Loading from SD 1
opti_angle 500.00
opti_latitude 481547833
opti_longtitude 169501500
opti_dist_m 0

Loading from SD 2
opti_angle 500.00
opti_latitude 483230666
opti_longtitude 171566000
opti_dist_m 0

w 1 481592832 169501488 0.00 500
w 1 481584256 169540992 36.00 500
w 1 481561728 169565408 72.00 499
w 1 481533952 169565408 108.00 499
w 1 481511456 169540992 144.00 499
w 1 481502880 169501488 180.00 499
w 1 481511456 169462000 216.00 499
w 1 481533952 169437584 252.00 499
w 1 481561728 169437584 288.00 500
w 1 481584256 169462000 324.00 500
Storing to SD 0
opti_angle 500.00
opti_latitude 481547833
opti_l

Loading from SD 6
opti_angle 324.00
opti_latitude 484048096
opti_longtitude 174991632
opti_dist_m 10628

w 5 483130272 174612000 314.00 18818
w 5 483125472 174605968 306.00 18816
w 5 483121504 174602128 300.00 18815
w 5 483118752 174599920 296.00 18815
w 5 483119424 174600432 297.00 18814
w 5 483118016 174599392 295.00 18814
w 5 483118016 174599392 295.00 18814
w 5 483119424 174600432 297.00 18814
w 5 483122176 174602720 301.00 18815
w 5 483126080 174606672 307.00 18817
w 5 483126720 174607376 308.00 18816
w 5 483131360 174613664 316.00 18819
Storing to SD 4
opti_angle 314.00
opti_latitude 482435776
opti_longtitude 174227584
opti_dist_m 9980

Loading from SD 7
opti_angle 500.00
opti_latitude 481773000
opti_longtitude 175786666
opti_dist_m 0

w 6 484042944 174982736 314.00 36435
w 6 484038112 174976688 306.00 36329
w 6 484034144 174972832 300.00 36243
w 6 484031424 174970624 296.00 36184
w 6 484029952 174969632 294.00 36152
w 6 484029952 174969632 294.00 36152
w 6 484031424 174970624 29

Loading from SD 6
opti_angle 313.50
opti_latitude 484042624
opti_longtitude 174982336
opti_dist_m 10497

w 5 483131168 174613376 315.65 18745
w 5 483131136 174613344 315.61 18744
w 5 483131136 174613328 315.58 18744
w 5 483131136 174613312 315.56 18744
w 5 483131104 174613280 315.55 18745
w 5 483131136 174613312 315.56 18744
w 5 483131136 174613312 315.56 18744
w 5 483131136 174613312 315.57 18744
w 5 483131136 174613328 315.59 18744
w 5 483131136 174613344 315.62 18744
w 5 483131168 174613376 315.66 18745
Storing to SD 4
opti_angle 313.66
opti_latitude 482435584
opti_longtitude 174227312
opti_dist_m 9959

Loading from SD 7
opti_angle 500.00
opti_latitude 481773000
opti_longtitude 175786666
opti_dist_m 0

w 6 484042624 174982304 313.45 36430
w 6 484042592 174982256 313.41 36429
w 6 484042592 174982240 313.38 36429
w 6 484042560 174982224 313.36 36428
w 6 484042560 174982224 313.35 36428
w 6 484042560 174982224 313.35 36428
w 6 484042560 174982224 313.36 36428
w 6 484042592 174982240 31

IndexError: list index out of range